# GPT-3.5 Classifier 

## Arousal (with examples)

In [3]:
import pandas as pd

In [1]:
# set directory to get .env file. You will need to change this to wherever your API key (.env file) is, as well as where you want to save your results
%cd "/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2" 

/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2


In [6]:
df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-Fiction-Full.csv') # load in the data using pandas and take a peek at the first 10 rows
df.head(10) #grab the text column 

,Unnamed: 0,gpt_formality_ratings,Text,file_name,year,month,gpt_coding_raw,gpt_coding_final
0,2476,1.0,The Story of the Queen The Story of the Queen ...,637338-the-story-of-the-queen.txt,1903,11,Fiction.,fiction
1,6773,0.0,Burn Your Maps Burn Your Maps Robyn Joy Leff A...,302385-burn-your-maps.txt,2002,1,fiction,fiction
2,6116,0.0,The Neurotic S Notebook The Neurotic S Noteboo...,658882-the-neurotic-s-notebook.txt,1962,6,The text is fiction.,fiction
3,3047,1.0,Their Wedding Journey Their Wedding Journey Wi...,630459-their-wedding-journey.txt,1871,8,fiction.,fiction
4,4092,1.0,De Grey : A Romance De Grey : A Romance IT was...,629196-de-grey-a-romance.txt,1868,7,fiction,fiction
5,3218,0.0,PS PS Jill McCorkle What I know now is that I ...,307540-ps.txt,2009,8,fiction,fiction
6,3768,1.0,Once I Lived Once I Lived Karl Kirchwey A poem...,559160-once-i-lived.txt,2018,6,The text is fiction.,fiction
7,3463,0.0,Shanghai Murmur Shanghai Murmur Te-Ping Chen A...,617269-shanghai-murmur.txt,2021,1,fiction,fiction
8,3414,0.0,Parents Parents Greg Delanty What do any of us...,308024-parents.txt,2010,5,fiction,fiction
9,1192,0.0,Journal of a Plague Year Journal of a Plague Y...,664183-journal-of-a-plague-year.txt,1973,8,fiction,fiction


In [ ]:
# Read in your examples as `examples` and take a peek at the dataframe
examples=pd.read_csv('/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/GPT-examples/Atlantic-Fiction-Examples.csv', index_col=0)
examples.head()

In [ ]:
inputs = df.Text.values #save the text as an object called inputs. we'll use this to run our observations through the pipeline

In [ ]:
# View the first 5 text values to check everything out 
print(inputs[:5])

In [ ]:
#load other dependencies 
import argparse 
import json
import os
import numpy as np
import openai
import pandas as pd
import random
import time
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import time

In [ ]:
title='BBPRIME'
subtitle='Arousal'
stim_set=title+'-'+subtitle
seed=1 #set seed to make sure we get reproducable results 
temperature=0.0 #want a low baking temp to have little variability or creativity, and so we can get the same results every. single. time. range is 0-1
engine='gpt-3.5-turbo' #change this to use different models available to you via OpenAI
n_context=1
cache = True   
resume=False
# MIDI='freq' #or 'name', 'number', 'freq'
audience='People'
item='Text'

load_dotenv()

apiKey = os.environ.get('steven_key') #for stevens key. update to you whatever you're calling your api_key. it should look like this in your file

# apiKey = sk-ZJ...................4T

openai.api_key = apiKey #installize the key so we can access

if cache:
    random.seed(seed)

In [ ]:
def generate_prompt(current):#(example_idxs, current_pair):
    prompt = f"""Your task is to classify a piece of as having either high or low arousal. Arousal is the extent to which a stimulus is calming or exciting. Examples of emotions with high
arousal are anger, joy, fear, surprise, or anxiety. Examples of emotions with low arousal are sadness, relaxation, disgust, content, or peace.

You will be provided with X examples of high arousal text and X examples of low text. Answer with only a whole number betwen 1 and 5, with 1 being
 low arousal and 5 being high arousal.


Text: {examples.Example.iloc[0]}  
Rating: {examples.Label.iloc[0]}
    
Text: {examples.Example.iloc[1]}
Rating: {examples.Label.iloc[1]}

Text: {examples.Example.iloc[2]}
Rating: {examples.Label.iloc[2]}

Text: {examples.Example.iloc[3]}
Rating: {examples.Label.iloc[3]}
    
Text: {inputs[current]}
Rating:
"""
    if n_context==0:
        prompt = f"""What is the arousal of following piece of text? Answer only with a number between 1 and 5: 1 if low arousal, and 5 if high arousal Here is the text:\n{inputs[current]}"""
    
    return prompt

cache_folder = f'cache/{stim_set}'
os.makedirs(cache_folder, exist_ok=True)
def create_cache_filename():
    filename = f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}'
    # if args.shuffle_context_each_draw:
    #     filename += '-shuffle'
    return os.path.join(cache_folder, filename + '.json')

if not resume:
    visited = []
    predicted_ratings = np.zeros((len(inputs)))
    request_count = 0

cache = {}

if cache and os.path.exists(create_cache_filename()):
    cache = json.load(open(create_cache_filename()))
cached_keys = list(cache.keys())

for idx1, bname1 in enumerate(tqdm(inputs)):
    current = idx1
    if current in visited and predicted_ratings[current]!=0:
        print(f'Already visited {current}')
        continue

    visited.append(current)

    key = f'{current}'
    if key in cached_keys:
        choices = cache[key]['choices']
        print('Using cached choices for key', key)
    else:
        prompt = generate_prompt(current)#(example_idxs, current_pair)
        response=False
        i=0
        while not response:
            i+=1
            try:
                response = openai.ChatCompletion.create(
                    model=engine,
                    messages=[{'role':'user', 'content':prompt}],
                    temperature=temperature,
                    timeout=10
                )
            except Exception as e:
                print(f'Attempt {i} failed')
                time.sleep(5)
        choices = [dict(choice.items()) for choice in response.choices]

        cache[key] = {
            'prompt': prompt,
            'choices': choices,
            'created': response.created
        }

        request_count += 1
        if cache and request_count % 5 == 0:
            json.dump(cache, open(create_cache_filename(), 'w'))

    try:
        answer = choices[0]['message']['content'].replace('\n', '').strip()
        predicted_ratings[idx1] = int(answer)
    except:
        print('Error', cache[key])

os.makedirs(f'predictions/{stim_set}', exist_ok=True)
np.save(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', predicted_ratings)


In [ ]:
df=pd.read_csv("/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/600-words-Atlantic-CSV-Formal-Informal-Fiction-Full.csv") #read in the OG dataframe again
predicted_ratings=np.load(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', allow_pickle=True)
df.head()

In [ ]:
predicted_ratings # just to check it ran smoothly
df.columns

In [ ]:
arousal_df_gpt35_250=pd.DataFrame(predicted_ratings, columns=['gpt_arousal_ratings'])
arousal_df_gpt35_250['Text']=df.Text.values #update to whatever the txt column is called
arousal_df_gpt35_250['SID']=df.file_name.values #update to SID column 

In [ ]:
#save the work
output_directory = '/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/output/final/' #update to where you want to save the work
os.makedirs(output_directory, exist_ok=True)

arousal_df_gpt35_250.to_csv(os.path.join(output_directory, f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}.csv'), index=False) #saving the pandas dataframe to a csv

## Emotional Tone (with examples)

In [17]:
import pandas as pd
    # set directory to get .env file

In [18]:
# set directory to get .env file. You will need to change this to wherever your API key (.env file) is, as well as where you want to save your results
%cd "/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2" 

/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2


In [32]:
df=pd.read_csv('/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/500-words-Atlantic-CSV-Formal-Informal-Fiction-Full.csv') # load in the data using pandas and take a peek at the first 10 rows
df.head(10) #grab the text column 

,Unnamed: 0,file_name,Text,year,month,gpt_coding_raw,gpt_coding_final,Tokens
0,35071,641130-the-wealth-we-wasted.txt,The Wealth We Wasted The Wealth We Wasted A. W...,1952,8,non-fiction,non-fiction,"['The', 'Wealth', 'We', 'Wasted', 'The', 'Weal..."
1,35072,641803-the-peripatetic-reviewer.txt,The Peripatetic Reviewer The Peripatetic Revie...,1952,8,Non-fiction.,non-fiction,"['The', 'Peripatetic', 'Reviewer', 'The', 'Per..."
2,35073,641018-a-life-of-disagreement.txt,A Life of Disagreement A Life of Disagreement ...,1952,8,Non-fiction.,non-fiction,"['A', 'Life', 'of', 'Disagreement', 'A', 'Life..."
3,35075,641902-spain.txt,Spain Spain on the World TodayDURING the riots...,1952,8,Non-fiction.,non-fiction,"['Spain', 'Spain', 'on', 'the', 'World', 'Toda..."
4,35076,642759-record-reviews.txt,Record Reviews Record Reviews John M. Conly by...,1952,8,Non-fiction.,non-fiction,"['Record', 'Reviews', 'Record', 'Reviews', 'Jo..."


In [36]:
# Tone examples 
examples=pd.read_csv('/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/GPT-examples/Atlantic-Nonfiction-Examples.csv') #update with the relevant CSV path
examples.head()

,file_name,year,Example,Label,formality_rating,gpt_coding_final,angela_formality_rating,lyle_formality_rating,ben_formality_rating,steven_formality_rating,interrater_reliability,overall_inter_rater_reliability,GPT3_5_NO_example,no_example_cor,GPT3_5_with_example,example_cor,GPT3_5_2_example
0,628188-the-question-of-the-hour.txt,1861,"However true this may be in the main, — and it...",1,NaN,non-fiction,1,1,1,1,1.000000,0.280369,1,NaN,1,NaN,1
1,633747-why-our-science-students-go-to-germany.txt,1889,Doubtless each individual is influenced by a v...,1,NaN,non-fiction,1,1,1,1,1.000000,0.280369,1,NaN,1,NaN,1
2,659289-why-im-a-cop-interviews-from-a-reporter...,1969,New York policeman for ten years; recently bec...,0,NaN,non-fiction,0,0,0,0,1.000000,0.280369,0,NaN,0,NaN,0
3,307617-how-american-health-care-killed-my-fath...,2009,"Like every grieving family member, I looked fo...",0,NaN,non-fiction,1,0,0,0,-0.222222,0.280369,0,NaN,0,NaN,0


In [37]:
inputs = df.Text.values #save the text
print(inputs[:3])

["The Wealth We Wasted The Wealth We Wasted A. W. Smith A. W. SMITH , who was born in Spain of British parents , lived in Russia as a boy and was educated at Shrewsbury and at Sandhurst . He served with the British Army throughout both World Wars . For some years he has made his home in Ipswich , Massachusetts , where he wrote his books , The Captain Departed and The Sword and the Rose , and a number of short stories which have appeared in the Atlantic . He has recently become an American citizen , and is today a Vice President on the staff of The Conservation Foundation . by A. W. SMITH1IT is probably near-heresy to suggest that the pioneer forefathers may not have been well equipped to deal intelligently with the problems posed in their new environment . Many of their practices , so evil and so wasteful in the coldly superior light of the middle of the twentieth century , may have been forced on them by the demands of the moment . But these are not sufficient to account for the destr

In [38]:
#load other dependencies 
import argparse
import json
import os
import numpy as np
import openai
import pandas as pd
import random
import time
from tqdm.notebook import tqdm
from dotenv import load_dotenv

In [39]:
title='BBPRIME'
subtitle='Arousal'
stim_set=title+'-'+subtitle
seed=1 #set seed to make sure we get reproducable results 
temperature=0.0 #want a low baking temp to have little variability or creativity, and so we can get the same results every. single. time. range is 0-1
engine='gpt-3.5-turbo' #change this to use different models available to you via OpenAI
n_context=1
cache = True   
resume=False
# MIDI='freq' #or 'name', 'number', 'freq'
audience='People'
item='Text'

load_dotenv()

apiKey = os.environ.get('steven_key') #for stevens key. update to you whatever you're calling your api_key. it should look like this in your file

# apiKey = sk-ZJ...................4T

openai.api_key = apiKey #installize the key so we can access

if cache:
    random.seed(seed)

In [ ]:
def generate_prompt(current):#(example_idxs, current_pair):
    prompt = f"""Your task is to classify the emptional tone of a piece of text positive or negative. Emotional tone is the degree of positivity or negativity of a text. Positive tone has a more upbeat
style and can include words like “good, well, new, love” but is not limited to these words. Negative tone reveals anxiety, sadness, and hostility and can include words like “bad, wrong, hate, too much” but is not limited to these words.
You will be provided with XX examples of text with a positive tone and XX examples of text with a negative tone. Answer with only a number, with 0 being negative tone and 1 being positive tone

Text: {examples.Example.iloc[0]} 
Rating: {examples.Label.iloc[0]}
    
Text: {examples.Example.iloc[1]}
Rating: {examples.Label.iloc[1]}

Text: {examples.Example.iloc[2]}
Rating: {examples.Label.iloc[2]}

Text: {examples.Example.iloc[3]}
Rating: {examples.Label.iloc[3]}
    
Text: {inputs[current]}
Rating:
"""
    if n_context==0:
        prompt = f"""Does this text have a positive or negative tone? Answer with only a number, with 0 being negative tone and 1 being positive tone. Here is the text:\n{inputs[current]}"""
    
    return prompt

cache_folder = f'cache/{stim_set}'
os.makedirs(cache_folder, exist_ok=True)

def create_cache_filename():
    filename = f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}'
    # if args.shuffle_context_each_draw:
    #     filename += '-shuffle'
    return os.path.join(cache_folder, filename + '.json')

if not resume:
    visited = []
    predicted_ratings = np.zeros((len(inputs)))
    request_count = 0

cache = {}

if cache and os.path.exists(create_cache_filename()):
    cache = json.load(open(create_cache_filename()))
cached_keys = list(cache.keys())

for idx1, bname1 in enumerate(tqdm(inputs)):
    current = idx1
    if current in visited and predicted_ratings[current] != 0:
        print(f'Already visited {current}')
        continue

    visited.append(current)

    key = f'{current}'
    if key in cached_keys:
        choices = cache[key]['choices']
        print('Using cached choices for key', key)
    else:
        prompt = generate_prompt(current)
        response = False
        i = 0
        while not response:
            i += 1
            try:
                response = openai.ChatCompletion.create(
                    model=engine,
                    messages=[{'role': 'user', 'content': prompt}],
                    temperature=temperature,
                    timeout=10
                )
            except Exception as e:
                print(f'Attempt {i} failed\n{e}\n')
                
                time.sleep(7) #7 second delay 
        choices = [dict(choice.items()) for choice in response.choices]

        cache[key] = {
            'prompt': prompt,
            'choices': choices,
            'created': response.created
        }

        request_count += 1
        if cache and request_count % 5 == 0:
            json.dump(cache, open(create_cache_filename(), 'w'))

    try:
        answer = choices[0]['message']['content'].replace('\n', '').strip()
        predicted_ratings[idx1] = int(answer)
    except Exception as e:
        print(f'Error: {e}\n\n{cache[key]}')

os.makedirs(f'predictions/{stim_set}', exist_ok=True)
np.save(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', predicted_ratings)


In [ ]:
df=pd.read_csv("/Users/stevenmesquiti/Dropbox/Collected Texts/Atlantic-csv/600-words-Atlantic-CSV-Formal-Informal-Fiction-Full.csv") #read in the OG dataframe again
predicted_ratings=np.load(f'predictions/{stim_set}/{stim_set}-{n_context}-{engine}-{temperature}-{seed}.npy', allow_pickle=True)
df.head()

In [42]:
predicted_ratings # just to check it ran smoothly
df.columns

Index(['Unnamed: 0', 'file_name', 'Text', 'year', 'month', 'gpt_coding_raw',
       'gpt_coding_final', 'Tokens'],
      dtype='object')

In [43]:
tone_df_gpt35=pd.DataFrame(predicted_ratings, columns=['tone_ratings'])
tone_df_gpt35['Text']=df.Text.values #update with text column 
tone_df_gpt35['SID']=df.file_name.values #update with indexing variable 

In [44]:
#save the work
output_directory = '/Users/stevenmesquiti/Desktop/working-with-lyle/Text-Annotation/blind-round-2/output/final' #update to your own directory 
os.makedirs(output_directory, exist_ok=True)
tone_df_gpt35.to_csv(os.path.join(output_directory, f'{stim_set}-{n_context}-{engine}-{temperature}-{seed}.csv'), index=False)